In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

Dataset 불러오기

In [ ]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv"
column_names = [
    'n_Pregnancies', 'Glucose_Concentration', 'Blood_Pressure(mmHg)',
    'Skin_Thickness(mm)', 'Serum_Insulin(mu_U/ml)', 'BMI',
    'Diabetes_Pedigree_Function', 'Age', 'Outcome'
]

# 1) CSV(URL)에서 데이터 읽기
df = pd.read_csv(url, names=column_names)  

# 2) 인슐린 컬럼 삭제
df = df.drop(columns=['Serum_Insulin(mu_U/ml)']) # 혈액 검사 이전의 당뇨병 예측 모델을 만들기 위해, 직접적인 인슐린 수치는 제외

# 3) 피클로 저장
df.to_pickle("diabetes.pkl") # 파이썬 객체 그대로 저장하고 다시 불러올 수 있다

print("원본 데이터:", df.shape)
print(df.head())
print("피클 파일 저장 완료 → diabetes.pkl")


원본 데이터: (768, 8)
   n_Pregnancies  Glucose_Concentration  Blood_Pressure(mmHg)  \
0              6                    148                    72   
1              1                     85                    66   
2              8                    183                    64   
3              1                     89                    66   
4              0                    137                    40   

   Skin_Thickness(mm)   BMI  Diabetes_Pedigree_Function  Age  Outcome  
0                  35  33.6                       0.627   50        1  
1                  29  26.6                       0.351   31        0  
2                   0  23.3                       0.672   32        1  
3                  23  28.1                       0.167   21        0  
4                  35  43.1                       2.288   33        1  
피클 파일 저장 완료 → diabetes.pkl


피클 내용 확인하기

In [3]:
df = pd.read_pickle("diabetes.pkl")

print("피클 로드:", df.shape)
print(df.head())

피클 로드: (768, 8)
   n_Pregnancies  Glucose_Concentration  Blood_Pressure(mmHg)  \
0              6                    148                    72   
1              1                     85                    66   
2              8                    183                    64   
3              1                     89                    66   
4              0                    137                    40   

   Skin_Thickness(mm)   BMI  Diabetes_Pedigree_Function  Age  Outcome  
0                  35  33.6                       0.627   50        1  
1                  29  26.6                       0.351   31        0  
2                   0  23.3                       0.672   32        1  
3                  23  28.1                       0.167   21        0  
4                  35  43.1                       2.288   33        1  


In [4]:
# 0이 하나라도 있는 컬럼 찾기
zero_columns = [col for col in df.columns if (df[col] == 0).any()]
print("0이 하나라도 있는 컬럼:", zero_columns)

0이 하나라도 있는 컬럼: ['n_Pregnancies', 'Glucose_Concentration', 'Blood_Pressure(mmHg)', 'Skin_Thickness(mm)', 'BMI', 'Outcome']


방법1) 결측치 제거

In [5]:
# 0이 나올 수 없는 생체 측정값을 Nan으로 결측치 처리하기
zero_cols = [
    'Glucose_Concentration', # 혈액 속에 포함된 포도당의 양
    'Blood_Pressure(mmHg)', # 확장기 혈압
    'Skin_Thickness(mm)', # 삼두근 피부 두겹 두께
    'BMI' # 체질량 지수
] # 모두 0이 나올 수 없는 수치이기에 결측치라고 판단

df[zero_cols] = df[zero_cols].replace(0, np.nan)

print(df.isnull().sum())

# NaN(결측치) 제거
df_clean = df.dropna()

print("결측치 제거 후 데이터:", df_clean.shape)
print(df_clean.head())

# 전처리 완료본 저장 (새 파일로)
df_clean.to_pickle("diabetes_dropna.pkl")
print("피클 저장 완료 → diabetes_dropna.pkl")

n_Pregnancies                   0
Glucose_Concentration           5
Blood_Pressure(mmHg)           35
Skin_Thickness(mm)            227
BMI                            11
Diabetes_Pedigree_Function      0
Age                             0
Outcome                         0
dtype: int64
결측치 제거 후 데이터: (532, 8)
   n_Pregnancies  Glucose_Concentration  Blood_Pressure(mmHg)  \
0              6                  148.0                  72.0   
1              1                   85.0                  66.0   
3              1                   89.0                  66.0   
4              0                  137.0                  40.0   
6              3                   78.0                  50.0   

   Skin_Thickness(mm)   BMI  Diabetes_Pedigree_Function  Age  Outcome  
0                35.0  33.6                       0.627   50        1  
1                29.0  26.6                       0.351   31        0  
3                23.0  28.1                       0.167   21        0  
4            

방법 1-1) 결측치 제거 로지스틱 회귀

In [6]:
# 피클 로드
df = pd.read_pickle("diabetes_dropna.pkl")

# x,y 분리
x = df.drop("Outcome", axis=1)
y = df["Outcome"]

# 훈련/테스트
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)  # train 데이터로 fit
x_test_scaled = scaler.transform(x_test)        # test 데이터는 transform만


# 모델 학습 (로지스틱 회귀)
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=2000)
model.fit(x_train, y_train)

#모델 평가
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

y_pred = model.predict(x_test)
y_proba = model.predict_proba(x_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))



Accuracy: 0.8037383177570093
ROC-AUC: 0.8744131455399061
[[65  6]
 [15 21]]
              precision    recall  f1-score   support

           0       0.81      0.92      0.86        71
           1       0.78      0.58      0.67        36

    accuracy                           0.80       107
   macro avg       0.80      0.75      0.76       107
weighted avg       0.80      0.80      0.80       107



방법 2) 결측치 중앙값으로 대체

In [7]:
# 1) 피클 로드
df = pd.read_pickle("diabetes.pkl")

# 2) 0이 나올 수 없는 컬럼 → NaN 처리
zero_cols = [
    'Glucose_Concentration',
    'Blood_Pressure(mmHg)',
    'Skin_Thickness(mm)',
    'BMI'
]
df[zero_cols] = df[zero_cols].replace(0, pd.NA)

# 3) 중앙값으로 NaN 대체 (안전하게, inplace 제거)
# 결측값 중앙값으로 대체
for col in zero_cols:
    df[col] = df[col].fillna(df[col].median())

print("결측치 처리 후 데이터:", df.shape)
print(df.head())

# 전처리 완료본 저장 (새 파일)
df.to_pickle("diabetes_median.pkl")
print("피클 저장 완료 → diabetes_median.pkl")

결측치 처리 후 데이터: (768, 8)
   n_Pregnancies  Glucose_Concentration  Blood_Pressure(mmHg)  \
0              6                  148.0                  72.0   
1              1                   85.0                  66.0   
2              8                  183.0                  64.0   
3              1                   89.0                  66.0   
4              0                  137.0                  40.0   

   Skin_Thickness(mm)   BMI  Diabetes_Pedigree_Function  Age  Outcome  
0                35.0  33.6                       0.627   50        1  
1                29.0  26.6                       0.351   31        0  
2                29.0  23.3                       0.672   32        1  
3                23.0  28.1                       0.167   21        0  
4                35.0  43.1                       2.288   33        1  
피클 저장 완료 → diabetes_median.pkl


/tmp/ipykernel_160531/319576972.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[col].median())
/tmp/ipykernel_160531/319576972.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[col].median())
/tmp/ipykernel_160531/319576972.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
 

방법 2-1) 결측치 중앙값 처리 로지스틱 회귀 모델

In [8]:
# 중앙값 처리 완료본 피클 로드
df = pd.read_pickle("diabetes_median.pkl")

# x,y 분리
x = df.drop("Outcome", axis=1)
y = df["Outcome"]

# 훈련/테스트
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)  # train 데이터로 fit
x_test_scaled = scaler.transform(x_test)        # test 데이터는 transform만


# 모델 학습 (로지스틱 회귀)
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=2000)
model.fit(x_train, y_train)

#모델 평가
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

y_pred = model.predict(x_test)
y_proba = model.predict_proba(x_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6883116883116883
ROC-AUC: 0.8122222222222223
Confusion Matrix:
 [[80 20]
 [28 26]]

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.80      0.77       100
           1       0.57      0.48      0.52        54

    accuracy                           0.69       154
   macro avg       0.65      0.64      0.64       154
weighted avg       0.68      0.69      0.68       154



방법 3) 평균으로 결측값 대체

In [9]:
# 평균으로 NaN 대체 (inplace 제거)
for col in zero_cols:
    df[col] = df[col].fillna(df[col].mean())

print("결측치 처리 후 행 수:", df.shape[0])
print("각 컬럼 결측치 합계:\n", df.isna().sum())

# 평균값 처리 완료본 피클 저장
df.to_pickle("diabetes_mean.pkl")
print("평균값 처리 완료 피클 저장됨: diabetes_mean.pkl")


# X, y 분리
x = df.drop("Outcome", axis=1)
y = df["Outcome"]

# 5) 훈련/테스트 분리
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 6) 로지스틱 회귀 학습
model = LogisticRegression(max_iter=2000)
model.fit(x_train, y_train)

# 7) 평가
y_pred = model.predict(x_test)
y_proba = model.predict_proba(x_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

결측치 처리 후 행 수: 768
각 컬럼 결측치 합계:
 n_Pregnancies                 0
Glucose_Concentration         0
Blood_Pressure(mmHg)          0
Skin_Thickness(mm)            0
BMI                           0
Diabetes_Pedigree_Function    0
Age                           0
Outcome                       0
dtype: int64
평균값 처리 완료 피클 저장됨: diabetes_mean.pkl
Accuracy: 0.6883116883116883
ROC-AUC: 0.8122222222222223
Confusion Matrix:
 [[80 20]
 [28 26]]

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.80      0.77       100
           1       0.57      0.48      0.52        54

    accuracy                           0.69       154
   macro avg       0.65      0.64      0.64       154
weighted avg       0.68      0.69      0.68       154



방법 4) KNN Imputer로 결측값 대체

In [10]:
# 1) 피클 로드
df = pd.read_pickle("diabetes.pkl")

# 2) 0이 나올 수 없는 컬럼 → NaN 처리 (np.nan 사용!)
zero_cols = [
    'Glucose_Concentration',
    'Blood_Pressure(mmHg)',
    'Skin_Thickness(mm)',
    'BMI'
]
df[zero_cols] = df[zero_cols].replace(0, np.nan)

# 3) KNNImputer로 결측치 대체
imputer = KNNImputer(n_neighbors=5)
df[zero_cols] = imputer.fit_transform(df[zero_cols])

# 4) X, y 분리
x = df.drop("Outcome", axis=1)
y = df["Outcome"]

# 5) 훈련/테스트 분리
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


# 6) 로지스틱 회귀 학습
model = LogisticRegression(max_iter=2000)
model.fit(x_train, y_train)

# 7) 평가
y_pred = model.predict(x_test)
y_proba = model.predict_proba(x_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.6883116883116883
ROC-AUC: 0.8131481481481482
Confusion Matrix:
 [[80 20]
 [28 26]]

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.80      0.77       100
           1       0.57      0.48      0.52        54

    accuracy                           0.69       154
   macro avg       0.65      0.64      0.64       154
weighted avg       0.68      0.69      0.68       154



방법 5) 결측치 가장 많은 칼럼 삭제 + 결측치도 제거

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

# 1) 데이터 로드
df = pd.read_pickle("diabetes.pkl")
print("데이터 로드:", df.shape)

# 2) 0값 있는 컬럼 확인 (결측치 대체를 위해)
zero_cols = [col for col in df.columns if (df[col] == 0).any()]
print("0 있는 컬럼:", zero_cols)

# 3) 결측치 제거
# 3-1) 결측치 가장 많은 컬럼 삭제
col_nan_count = df[zero_cols].isna().sum()
drop_col = col_nan_count.idxmax()
df.drop(columns=drop_col, inplace=True)
print(f"가장 결측치 많은 컬럼 삭제: {drop_col}")

# 3-2) 나머지 결측치 행 삭제
df.dropna(inplace=True)
print("결측치 제거 후 데이터:", df.shape)

# 4) X, y 분리
X = df.drop(columns="Outcome")
y = df["Outcome"]

# 5) Train/Test 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 6) 로지스틱 회귀 학습
model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

# 7) 모델 평가
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


데이터 로드: (768, 8)
0 있는 컬럼: ['n_Pregnancies', 'Glucose_Concentration', 'Blood_Pressure(mmHg)', 'Skin_Thickness(mm)', 'BMI', 'Outcome']
가장 결측치 많은 컬럼 삭제: n_Pregnancies
결측치 제거 후 데이터: (768, 7)
Accuracy: 0.7272727272727273
ROC-AUC: 0.805
Confusion Matrix:
 [[84 16]
 [26 28]]

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.84      0.80       100
           1       0.64      0.52      0.57        54

    accuracy                           0.73       154
   macro avg       0.70      0.68      0.69       154
weighted avg       0.72      0.73      0.72       154



In [11]:
import os

# 현재 작업 디렉토리 확인
print(os.getcwd())

# 피클 파일만 필터링
files = [f for f in os.listdir('.') if f.endswith('.pkl')]
print("현재 디렉토리 피클 파일:", files)


/mnt/c/Users/KDT45/Downloads/Microsoft VS Code
현재 디렉토리 피클 파일: ['diabetes.pkl', 'diabetes_dropna.pkl', 'diabetes_mean.pkl', 'diabetes_median.pkl']


In [12]:
# dropna 시 행 수
df_dropna = df.dropna()
print("Dropna 데이터 행 수:", df_dropna.shape[0])

# median 대체 시 결측치 개수
df_median = df.copy()
for col in zero_cols:
    df_median[col] = df_median[col].fillna(df_median[col].median())
print("Median 대체 후 결측치 합계:", df_median.isnull().sum().sum())

Dropna 데이터 행 수: 768
Median 대체 후 결측치 합계: 0
